In [115]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [116]:
# get html
response = requests.get('https://worldpopulationreview.com/countries/countries-by-gdp')

#print status code
print('Status code:', response.status_code, end= '\n')

Status code: 200


In [117]:
#Create soup object
html = response.content
html = BeautifulSoup(html, 'html.parser') 

# display markup 
#html.prettify()

# selecting element
gdp_data_table = html.find(name='table', attrs={'class' : 'jsx-364991517 table is-striped is-hoverable is-fullwidth tp-table-body is-narrow'})
gdp_data_row = gdp_data_table.find(name='tbody', attrs={'class':'jsx-364991517'}).find_all(name = 'tr')

#print html
# gdp_data_row


In [118]:
# creates empty list
gdp_datapoints = list()

#Extract data
for row in gdp_data_row:
    data = row.find_all('td')

    data = [td.contents[0] if len(td.contents) == 1 else 'NAN' for td in data] 
    data[1] = data[1].contents[0]
        
    country, gdp_imf_20, gdp_un_16, gdp_per_capita, population = data[1:]

    gdp_data = {
        "country" : country,
        "gdp(imf '20)" : gdp_imf_20,
        "gdp per capita" : gdp_per_capita,
        "population" : population
    } 

    gdp_datapoints.append(gdp_data)


# print(gdp_datapoints)

    

In [123]:
# create dataframe
df_gdp = pd.DataFrame(gdp_datapoints)
df_gdp.index = df_gdp.index + 1

df_gdp

,country,gdp(imf '20),gdp per capita,population
1,United States,20.89 Tn,"$61,763","338,289,857"
2,China,14.86 Tn,"$10,423","1,425,887,337"
3,Japan,5.04 Tn,"$40,662","123,951,692"
4,Germany,3.84 Tn,"$46,100","83,369,843"
5,United Kingdom,2.76 Tn,"$40,867","67,508,936"
...,...,...,...,...
208,New Caledonia,NAN,"$32,577","289,950"
209,Anguilla,NAN,"$21,285","15,857"
210,Sudan,NAN,"$1,768","46,874,204"
211,Syria,NAN,"$1,002","22,125,249"
